# Predicción de postulación para un aviso

In [ ]:
import numpy as np  
import pandas as pd
import gc
import datetime
import re
from math import sqrt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib
import matplotlib.pyplot as plt

In [ ]:
SAMPLE_SIZE = 1500000

## Carga y limpieza de datos / Feature Engineering

### Postulantes

In [ ]:
# cargo postulantes
df_postulantes1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])
df_postulantes2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])
df_postulantes3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv', parse_dates=['fechanacimiento'])

df_postulantes = df_postulantes1.append(df_postulantes2).append(df_postulantes3)

del df_postulantes1
del df_postulantes2
del df_postulantes3

df_postulantes.drop_duplicates(['idpostulante'], keep='first', inplace=True)

df_postulantes.shape

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# guardamos los codificadores (label => numero y visceversa) en un diccionario
label_encoders = {}

In [ ]:
# limpieza de datos de fecha de nacimiento
df_postulantes['fechanacimiento'] = pd.to_datetime(df_postulantes['fechanacimiento'], errors='coerce')

df_postulantes['edad'] = datetime.datetime.now().year - df_postulantes['fechanacimiento'].dt.year
df_postulantes['edad'] = df_postulantes['edad'].fillna(0)

df_postulantes = df_postulantes.drop(['fechanacimiento'], axis=1)

df_postulantes = df_postulantes.loc[(df_postulantes['sexo'] == 'FEM') | (df_postulantes['sexo'] == 'MASC') | (df_postulantes['sexo'] == 'NO_DECLARA')]

# convierto variables categóricas a numéricas
label_encoders['sexo'] = LabelEncoder().fit(['FEM', 'MASC', 'NO_DECLARA'])
df_postulantes['sexo'] = label_encoders['sexo'].transform(df_postulantes['sexo'])

print(df_postulantes.shape)
print(df_postulantes.head())

### Educacion

In [ ]:
# cargo educacion de los estudiantes
df_edu1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_1_postulantes_educacion.csv')
df_edu2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
df_edu3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv')

df_edu = df_edu1.append(df_edu2).append(df_edu3)

del df_edu1
del df_edu2
del df_edu3
gc.collect()

print(df_edu.shape)
print(df_edu.head())

In [ ]:
# renombro columnas para no confundirlas luego de mergear
df_edu = df_edu.rename(columns={'nombre':'nombre_edu', 'estado': 'estado_edu'});

In [ ]:
# convierto variables categóricas a numéricas
label_encoders['nombre_edu'] = LabelEncoder().fit(df_edu['nombre_edu'])
label_encoders['estado_edu'] = LabelEncoder().fit(df_edu['estado_edu'])

df_edu['nombre_edu'] = label_encoders['nombre_edu'].transform(df_edu['nombre_edu'])
df_edu['estado_edu'] = label_encoders['estado_edu'].transform(df_edu['estado_edu'])

df_edu.head()

In [ ]:
df_posts_edu = df_postulantes.merge(df_edu, on='idpostulante', how='left')

df_posts_edu['nombre_edu'] = df_posts_edu['nombre_edu'].fillna(-1)
df_posts_edu['estado_edu'] = df_posts_edu['estado_edu'].fillna(-1)

# me quedo con el de mayor educacion registro para cada postulante
df_posts_edu.sort_values(by='nombre_edu', ascending=False)

df_posts_edu.drop_duplicates(subset = "idpostulante",keep= "first", inplace=True)

print(df_posts_edu.shape)
print(df_posts_edu.head())

del df_edu
del df_postulantes
gc.collect()

### Postulaciones

In [ ]:
# cargo postulaciones
df_postulaciones1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'])
df_postulaciones2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'])

df_postulaciones = df_postulaciones1.append(df_postulaciones2)

del df_postulaciones1
del df_postulaciones2
gc.collect()

df_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], keep='first', inplace=True)

print(df_postulaciones.shape)
print(df_postulaciones.head())

In [ ]:
df_postulaciones.drop('fechapostulacion', axis=1, inplace=True)

### Avisos

In [ ]:
# cargo avisos
df_avisos1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_6_avisos_detalle.csv')
df_avisos2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')
df_avisos3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv')
df_avisos4 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

df_avisos = df_avisos1.append(df_avisos2).append(df_avisos3).append(df_avisos4)

del df_avisos1
del df_avisos2
del df_avisos3
del df_avisos4
gc.collect()

df_avisos = df_avisos.drop_duplicates(['idaviso'], keep='first')

print(df_avisos.shape)
print(df_avisos.head())

#### Avisos online

In [ ]:
# cargo avisos
df_avisos_online1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_5_avisos_online.csv')
df_avisos_online2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv')

df_avisos_online = df_avisos_online1.append(df_avisos_online2)

del df_avisos_online1
del df_avisos_online2
gc.collect()

df_avisos_online = df_avisos_online.drop_duplicates(['idaviso'], keep='first')
df_avisos_online['online'] = 1

print(df_avisos_online.shape)
print(df_avisos_online.head())

In [ ]:
df_avisos = df_avisos.merge(df_avisos_online, how='left', on='idaviso')

del df_avisos_online
gc.collect()

In [ ]:
df_avisos = df_avisos.drop(['mapacalle'], axis=1)

In [ ]:
# limpieza de NaN, nan, None, etc.
df_avisos['ciudad'] = df_avisos['ciudad'].fillna('None')
df_avisos['titulo'] = df_avisos['titulo'].fillna('None')
df_avisos['descripcion'] = df_avisos['descripcion'].fillna('None')
df_avisos['denominacion_empresa'] = df_avisos['denominacion_empresa'].fillna('None')
df_avisos['nivel_laboral'] = df_avisos['nivel_laboral'].fillna('None')

df_avisos['online'] = df_avisos['online'].fillna(0)

In [ ]:
# convierto variables categóricas a numéricas
label_encoders['nombre_zona'] = LabelEncoder().fit(df_avisos['nombre_zona'])
label_encoders['ciudad'] = LabelEncoder().fit(df_avisos['ciudad'])
label_encoders['tipo_de_trabajo'] = LabelEncoder().fit(df_avisos['tipo_de_trabajo'])
label_encoders['nivel_laboral'] = LabelEncoder().fit(df_avisos['nivel_laboral'])
label_encoders['nombre_area'] = LabelEncoder().fit(df_avisos['nombre_area'])
label_encoders['denominacion_empresa'] = LabelEncoder().fit(df_avisos['denominacion_empresa'])

df_avisos['nombre_zona'] = label_encoders['nombre_zona'].transform(df_avisos['nombre_zona'])
df_avisos['ciudad'] = label_encoders['ciudad'].transform(df_avisos['ciudad'])
df_avisos['tipo_de_trabajo'] = label_encoders['tipo_de_trabajo'].transform(df_avisos['tipo_de_trabajo'])
df_avisos['nivel_laboral'] = label_encoders['nivel_laboral'].transform(df_avisos['nivel_laboral'])
df_avisos['nombre_area'] = label_encoders['nombre_area'].transform(df_avisos['nombre_area'])
df_avisos['denominacion_empresa'] = label_encoders['denominacion_empresa'].transform(df_avisos['denominacion_empresa'])

In [ ]:
df_avisos.head()

#### Trabajando el texto/titulo de los avisos

In [ ]:
regex_limpiar_html_tags = re.compile('<.*?>')
def limpiar_html(strhtml):
    return re.sub(regex_limpiar_html_tags, '', strhtml)

def regularizar_texto(linea):
    return limpiar_html(linea)\
                        .lower()\
                        .replace('á', 'a')\
                        .replace('é', 'e')\
                        .replace('í', 'i')\
                        .replace('ó', 'o')\
                        .replace('ú', 'u')\
                        .replace('\t', '')\
                        .replace('\n', '')\
                        .replace('\r', '')
                        
vregularizar_texto = np.vectorize(regularizar_texto)

columnas_terminos = {
#    'ingenieria': ['ingeniero', 'ingeniera', 'ingenieria'],
#    'software': ['javascript', 'java', 'html', 'css', 'c#', '.net', 'android', 'ios', 'php', 'c++', 'sql', 'it resources'],
#    'lunes_a_viernes': ['lunes a viernes', 'lun a vier', 'lun a vie'],
#    'requiere_titulo': ['titulo secundario', 'titulo terciario', 'titulo universitario', 'secundario completo', 'estudios completo', 'universitarios completo'],
#    'marketing': ['marketing', 'telemarketer', 'telemarketing', 'marketer', 'media manager', 'callcenter', 'call center'],
#    'capacitacion': ['capacitacion'],
#    'idioma_ingles': ['idioma ingles', 'manejo de ingles', 'clases de ingles', 'ingles excluyente', 'ingles requerido'],
#    'multinacional': ['multinacional'],
#    'internacional': ['internacional'],
#    'atencion_al_cliente': ['atencion al cliente', 'call center', 'callcenter', 'soporte tecnico', 'area de soporte', 'tareas de soporte'],
#    'turismo': ['turismo'],
#    'zona_puerto_madero': ['puerto madero'],
#    'zona_centro': ['microcentro', 'tribunales'],
#    'experiencia_previa': ['experiencia previa', 'experiencias anteriores', 'años de experiencia'],
#    'obra_social': ['obra social', 'osde', 'swiss medical', 'galeno', 'wh hope', 'grupo familiar', 'cobertura medica', 'pre paga', 'prepaga'],
#    'puesto_gerencia': ['gerente', 'gerenta', 'gerencia'],
#    'requisitos_excluyentes': ['excluyente'],
#    'retail': ['hipermercado', 'supermercado', 'cadena', 'franquicia', 'fravega', 'retail', 'vendedor'],
#    'chofer': ['chofer', 'taxi', 'remis', 'colectivo', 'reparto', 'furgon', 'camion'],
#    'medicina': ['medic', 'hospital', 'clinica', 'farmacia']
}

def tiene_termino(texto1, texto2, terminos):
    for t in terminos:
        if t in texto1 or t in texto2:
            return 1
    return 0
def vtiene_termino(serie1, serie2, terminos):
    if len(serie1) != len(serie2):
        raise ValueError('series de distinto largo')
    s = []
    for i in range(0, len(serie1)):
        s.append(tiene_termino(serie1.iloc[i], serie2.iloc[i], terminos))
    return pd.Series(s)

In [ ]:
df_avisos['titulo'] = vregularizar_texto(df_avisos['titulo'])
df_avisos['descripcion'] = vregularizar_texto(df_avisos['descripcion'])

In [ ]:
# optimizable, podríamos aplicar para cada fila todas las columnas del diccionario
# como está ahora hace k*n con k=|columnas_terminos| y n=|df_avisos|
for col, terminos in columnas_terminos.items():
    df_avisos[col] = vtiene_termino(df_avisos['titulo'], df_avisos['descripcion'], terminos)

In [ ]:
for col in columnas_terminos:
    print("col = %s" % col)
    print(df_avisos[col].value_counts())

In [ ]:
df_avisos.drop(['titulo', 'descripcion'], axis=1, inplace=True)

In [ ]:
#cargo tf idf 
df_avisos_tfidf = pd.read_csv("./kevin/df_aviso_svd.csv")
df_avisos= df_avisos.merge(df_avisos_tfidf, on= "idaviso")

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
n = int(sqrt(df_avisos.shape[0]))
kmeans = KMeans(n_clusters=n)
kmeans.fit(df_avisos_tfidf)
df_avisos['cluster'] = kmeans.labels_

In [ ]:
df_avisos['cluster'].value_counts()

In [ ]:
del kmeans
del df_avisos_tfidf
gc.collect()

### Vistas

In [ ]:
# cargo avisos
df_vistas1 = pd.read_csv('../datos_navent_fiuba/datos_navent/fiuba_3_vistas.csv', parse_dates=['timestamp'])
df_vistas2 = pd.read_csv('../datos_navent_fiuba/fiuba_hasta_15_abril/fiuba_3_vistas.csv', parse_dates=['timestamp'])
df_vistas3 = pd.read_csv('../datos_navent_fiuba/fiuba_desde_15_abril/fiuba_3_vistas.csv', parse_dates=['timestamp'])

df_vistas = df_vistas1.append(df_vistas2).append(df_vistas3)

del df_vistas1
del df_vistas2
del df_vistas3
gc.collect()

df_vistas = df_vistas.rename(columns={'idAviso':'idaviso'})
df_vistas = df_vistas.drop_duplicates(['idpostulante', 'idaviso'], keep='first')
gc.collect()

print(df_vistas.shape)
print(df_vistas.head())

In [ ]:
df_vistas['visto'] = 1
df_vistas.head()

### Vinculamos postulantes y avisos

In [ ]:
df_postulaciones_merge = df_postulaciones.sample(SAMPLE_SIZE)

# merge de todos los datos
df_general = df_posts_edu.merge(df_postulaciones_merge, on='idpostulante').merge(df_avisos, on='idaviso')

del df_postulaciones_merge
gc.collect()

print(df_general.shape)
print(df_general.head())

## Preparación de datos para entrenamiento y predicción

#### Generación de postulaciones

In [ ]:
df_general['sepostulo'] = 1

#### Generación de "no" postulaciones

In [ ]:
sample = SAMPLE_SIZE

In [ ]:
df_postulantes_sample = df_posts_edu.sample(sample, replace=True).reset_index().drop("index",1)
df_avisos_sample = df_avisos.sample(sample, replace=True).reset_index().drop("index",1)

print(df_postulantes_sample.shape)
print(df_avisos_sample.shape)

In [ ]:
df_no_postulaciones = df_postulantes_sample.join(df_avisos_sample)

del df_postulantes_sample
del df_avisos_sample
gc.collect()

df_no_postulaciones = df_no_postulaciones.merge(df_postulaciones, on=["idaviso","idpostulante"], how="left")
df_no_postulaciones.drop_duplicates(['idaviso', 'idpostulante'], keep='first', inplace=True)
print(df_no_postulaciones.shape)

In [ ]:
del df_postulaciones
gc.collect()

In [ ]:
df_no_postulaciones['sepostulo'] = 0;
df_no_postulaciones.head(5)

In [ ]:
print(df_general.shape)
df_general = df_general.append(df_no_postulaciones)
print(df_general.shape)

In [ ]:
del df_no_postulaciones
gc.collect()

In [ ]:
df_general = df_general.merge(df_vistas, on=['idaviso', 'idpostulante'], how='left')
df_general['visto'] = df_general['visto'].fillna(0)

In [ ]:
# comentar si se quiere probar contra los dats de la competencia
del df_posts_edu
del df_avisos
del df_vistas
gc.collect()

In [ ]:
# comentar si se quiere probar contra los dats de la competencia
df_general = shuffle(df_general, random_state=13).reset_index()

gc.collect()

In [ ]:
# comentar si se quiere probar contra los dats de la competencia
offset = int(df_general.shape[0] * 0.8)

df_general_entrenamiento = df_general.loc[:offset]
df_general_test = df_general.loc[offset:]

In [ ]:
# comentar si se quiere probar contra los dats de la competencia
del df_general
gc.collect()

## Ejecución del algoritmo de ML

In [ ]:
columnas_datos = ['sexo', 'edad', 'nombre_edu', 'estado_edu', 'idpais', 'nombre_zona', 'ciudad', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'denominacion_empresa', 'online', 'visto', '0', '1', '2', '3', '4', '5', 'cluster'] + list(columnas_terminos.keys())
columnas_target = ['sepostulo']

In [ ]:
def guardar_res(df_res, predicciones, algoritmo, clasificador):
    now = datetime.datetime.now()
    filename = "./test_submissions/{0}-{1}-{2}.csv".format(algoritmo, now.date(), now.time())
    
    df_res['sepostulo'] = predicciones
    df_res.to_csv(filename, index=False)
    
    filename = "./test_models/{0}-{1}-{2}.pkl".format(algoritmo, now.date(), now.time())
    joblib.dump(clasificador, filename)
    
def plot_importances(classifier):
    imps = classifier.feature_importances_
    indices = np.argsort(imps)[::-1][:10] # invierto el orden y tomo 10

    x_labels = []
    y_vals = []
    for i in indices:
        x_labels.append(columnas_datos[i])
        y_vals.append(imps[i])

    numeric_x_labels = range(0, len(y_vals))
    plt.bar(numeric_x_labels, y_vals)
    plt.xticks(numeric_x_labels, x_labels, rotation=75);

In [ ]:
predicciones = []

### XGBoost

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
params = {'n_estimators': 100, 'learning_rate': 0.001 }#, 'max_depth': a, 'min_samples_split': 2, }

xgboostclassifier = GradientBoostingClassifier(**params)
xgboostclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target].values.ravel())

sepostulo_predicciones_xgb = xgboostclassifier.predict(df_general_test[columnas_datos])

plot_importances(xgboostclassifier)

prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones_xgb)
print("Precision %.4f" % (prec * 100))

In [ ]:
del xgboostclassifier
gc.collect()

### Random Forest

In [ ]:
# descartamos la columan 'cluster' porque no da buenos resultados para RF
columnas_datos = ['sexo', 'edad', 'nombre_edu', 'estado_edu', 'idpais', 'nombre_zona', 'ciudad', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'denominacion_empresa', 'online', 'visto', '0', '1', '2', '3', '4', '5'] + list(columnas_terminos.keys())

In [ ]:
#params = { 'n_estimators':300,'max_depth':17, 'random_state': 45, "max_features" : 0.320417} #0.320417 #0.125 creo que era
params = { 'n_estimators': 30, 'max_depth': 17, 'random_state': 45, "max_features": 0.320417 }

rndforestclassifier = RandomForestClassifier(**params)
rndforestclassifier.fit(df_general_entrenamiento[columnas_datos], df_general_entrenamiento[columnas_target].values.ravel())
sepostulo_predicciones_rf = rndforestclassifier.predict(df_general_test[columnas_datos])
             
plot_importances(rndforestclassifier)

prec = precision_score(df_general_test[columnas_target], sepostulo_predicciones_rf)
print("Precision %.4f" % (prec * 100))

In [ ]:
del rndforestclassifier
gc.collect()

### Ponderación

In [ ]:
predicciones_ponderadas = []

for i in range (0, len(sepostulo_predicciones_xgb)):
    predicciones_ponderadas.append((0.99 * sepostulo_predicciones_xgb[i]) + (0.01 * sepostulo_predicciones_rf[i]))
    
prec = precision_score(df_general_test[columnas_target], predicciones_ponderadas)
#print("Precision %.4f" % (prec * 100))
guardar_res(df_resultado, predicciones_ponderadas, "promedio_ponderado", None)

In [ ]:
del df_general_entrenamiento
del df_general_test

### Usando datos de prueba

In [ ]:
#df_test_final = pd.read_csv('../datos_navent_fiuba/test_final_100k.csv')
#print(df_test_final.shape)

#df_test_final = df_test_final.merge(df_posts_edu, on='idpostulante')
#print(df_test_final.shape)
#del df_posts_edu
#gc.collect()

#df_test_final = df_test_final.merge(df_avisos, on='idaviso')
#print(df_test_final.shape)
#del df_avisos
#gc.collect()

#df_test_final = df_test_final.merge(df_vistas, on=['idaviso', 'idpostulante'], how='left')
#df_test_final['visto'] = df_test_final['visto'].fillna(0)
#print(df_test_final.shape)
#del df_vistas
#gc.collect()

In [ ]:
#df_resultado = pd.DataFrame()
#df_resultado['id'] = df_test_final['id']

In [ ]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
#params = {'n_estimators': 100, 'learning_rate': 0.001 }#, 'max_depth': a, 'min_samples_split': 2, }

#xgboostclassifier = GradientBoostingClassifier(**params)
#xgboostclassifier.fit(df_general[columnas_datos], df_general[columnas_target].values.ravel())

#sepostulo_predicciones_proba = xgboostclassifier.predict_proba(df_test_final[columnas_datos])[:,1]

#guardar_res(df_resultado, sepostulo_predicciones_proba, "XGBoost", xgboostclassifier)